In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
import numpy as np
import csv
from sklearn.ensemble import RandomForestRegressor
import scipy
from scipy.stats.stats import pearsonr
from scipy import stats
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics

In [2]:

all_data=pd.read_csv('./data/train.csv')
##replace NaN with column mean
all_data.fillna(all_data.mean(),inplace = True) 

#df = all_data.drop(["id","opened_position_qty " ,"closed_position_qty" ,"transacted_qty","d_open_interest","bid1","bid2","bid3","bid4","bid5","ask1","ask2","ask3","ask4","ask5"],axis=1)
all_data["bid1sum"]=all_data["bid1"]*all_data["bid1vol"]
all_data["bid2sum"]=all_data["bid2"]*all_data["bid2vol"]
all_data["bid3sum"]=all_data["bid3"]*all_data["bid3vol"]
all_data["bid4sum"]=all_data["bid4"]*all_data["bid4vol"]
all_data["bid5sum"]=all_data["bid5"]*all_data["bid5vol"]
all_data["ask1sum"]=all_data["ask1"]*all_data["ask1vol"]
all_data["ask2sum"]=all_data["ask2"]*all_data["ask2vol"]
all_data["ask3sum"]=all_data["ask3"]*all_data["ask3vol"]
all_data["ask4sum"]=all_data["ask4"]*all_data["ask4vol"]
all_data["ask5sum"]=all_data["ask5"]*all_data["ask5vol"]


##downsizing for test
all_data=all_data.sample(frac=0.01, random_state=1)

train_dataset = all_data.sample(frac=0.8, random_state=1)
test_dataset = all_data.drop(train_dataset.index)

Train = train_dataset.to_numpy()
Test = test_dataset.to_numpy()
    
X_train = Train[:,1:-1]
Y_train = Train[:,-1]
print('y=1', np.sum(Y_train, axis=0))

X_test = Test[:,1:-1]
Y_test = Test[:,-1]


# Applying feature scaling on the train data  
min_max_scaler = MinMaxScaler()
X_test = min_max_scaler.fit_transform(X_test)
X_train = min_max_scaler.fit_transform(X_train)
print('X test',X_test[1])

C_vals=[1,10,1e2,1e3,1e4,1e5]
gamma_vals=[1e-12, 1e-9, 1e-6, 1e-3, 1,1e2]
for i in range(1):
    for j in range(1):
        #C_val=C_vals[i]
        C_val=1
        #gamma_val=gamma_vals[j]
        gamma_val=0.001
        print('=============================')
        print(C_val,gamma_val)

        classifier = SVR(kernel = 'rbf', C=C_val, gamma=gamma_val)
        
        # Fitting Linear Kernel SVM to the Training set
        # classifier = SVC(kernel = 'linear', random_state = 0)
        
        classifier.fit(X_train, Y_train)    
        prediction = classifier.predict(X_test)
        print(prediction)
        loss = np.sum(abs(prediction-Y_test),axis = 0)
 
        print(1-loss/len(Y_test))


        vali_data=pd.read_csv('./data/test.csv')
        vali_data.fillna(all_data.mean(),inplace = True) 
        #vali_data=vali_data.drop(["id","opened_position_qty " ,"closed_position_qty" ,"transacted_qty","d_open_interest","bid1","bid2","bid3","bid4","bid5","ask1","ask2","ask3","ask4","ask5"],axis=1)
        vali_data["bid1sum"]=vali_data["bid1"]*vali_data["bid1vol"]
        vali_data["bid2sum"]=vali_data["bid2"]*vali_data["bid2vol"]
        vali_data["bid3sum"]=vali_data["bid3"]*vali_data["bid3vol"]
        vali_data["bid4sum"]=vali_data["bid4"]*vali_data["bid4vol"]
        vali_data["bid5sum"]=vali_data["bid5"]*vali_data["bid5vol"]
        vali_data["ask1sum"]=vali_data["ask1"]*vali_data["ask1vol"]
        vali_data["ask2sum"]=vali_data["ask2"]*vali_data["ask2vol"]
        vali_data["ask3sum"]=vali_data["ask3"]*vali_data["ask3vol"]
        vali_data["ask4sum"]=vali_data["ask4"]*vali_data["ask4vol"]
        vali_data["ask5sum"]=vali_data["ask5"]*vali_data["ask5vol"]
        
        vali_data=vali_data.to_numpy()
        vali_data_id=vali_data[:,0]
        vali_data=vali_data[:,1:]

        vali_predict=classifier.predict(vali_data)      
        print(vali_predict[1:30])
        
        vali_df = pd.DataFrame({"id" : vali_data_id, "Predicted" : vali_predict})
        vali_df['id'] = vali_df['id'].astype(int)
        vali_df.to_csv("submission2.csv", index=False)
        

y=1 112467722.6
X test [0.71322034 0.71225457 0.06669445 0.0892853  0.         0.58333333
 0.71225457 0.71225457 0.71370421 0.71370421 0.71370421 0.71225457
 0.71205962 0.71186441 0.71254237 0.71254237 0.04878049 0.1025641
 0.01886792 0.         0.02739726 0.         0.05263158 0.
 0.02020202 0.01204819 0.         0.04964193 0.10894998 0.02088447
 0.00145456 0.02917758 0.00158062 0.05568534 0.00099343 0.0215059 ]
1 0.001
[15624.9601327  15625.16665253 15625.02835104 ... 15624.99392718
 15625.21698274 15625.66042707]
-15403.74110426831
[15625.09834639 15625.09834639 15625.09834639 15625.09834639
 15625.09834639 15625.09834639 15625.09834639 15625.09834639
 15625.09834639 15625.09834639 15625.09834639 15625.09834639
 15625.09834639 15625.09834639 15625.09834639 15625.09834639
 15625.09834639 15625.09834639 15625.09834639 15625.09834639
 15625.09834639 15625.09834639 15625.09834639 15625.09834639
 15625.09834639 15625.09834639 15625.09834639 15625.09834639
 15625.09834639]
